In [1]:
global Path
if sc.master[0:5]=="local":
    Path="file:///home/vmauser/pythonwork/PythonProject/"
else:
    Path="file:/home/vmauser/"

### Data preprocessing

#### Importing data

In [2]:
covtype_df=sqlContext.read.format("csv").option("header", "true").option(
    "delimiter", ",").load(Path+"data/covtype.data")

print(covtype_df.count())
print(len(covtype_df.columns))
print(covtype_df.columns)
covtype_df.printSchema()

581011
55
['2596', '51', '3', '258', '04', '510', '221', '232', '148', '6279', '110', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '142', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', '5']
root
 |-- 2596: string (nullable = true)
 |-- 51: string (nullable = true)
 |-- 3: string (nullable = true)
 |-- 258: string (nullable = true)
 |-- 04: string (nullable = true)
 |-- 510: string (nullable = true)
 |-- 221: string (nullable = true)
 |-- 232: string (nullable = true)
 |-- 148: string (nullable = true)
 |-- 6279: string (nullable = true)
 |-- 110: string (nullable = true)
 |-- 011: string (nullable = true)
 |-- 012: string (nullable = true)
 |-- 013: string (nullable = true)
 |-- 014: string (nullable = true)
 |-- 015: string (nullable = true)
 |-- 016: string (nullable = true)
 |-- 017: 

#### Type transformation

In [3]:
from pyspark.sql.functions import col

covtype_df=covtype_df.select([col(column).cast("double").alias(column) 
                             for column in covtype_df.columns])

print covtype_df.printSchema()

root
 |-- 2596: double (nullable = true)
 |-- 51: double (nullable = true)
 |-- 3: double (nullable = true)
 |-- 258: double (nullable = true)
 |-- 04: double (nullable = true)
 |-- 510: double (nullable = true)
 |-- 221: double (nullable = true)
 |-- 232: double (nullable = true)
 |-- 148: double (nullable = true)
 |-- 6279: double (nullable = true)
 |-- 110: double (nullable = true)
 |-- 011: double (nullable = true)
 |-- 012: double (nullable = true)
 |-- 013: double (nullable = true)
 |-- 014: double (nullable = true)
 |-- 015: double (nullable = true)
 |-- 016: double (nullable = true)
 |-- 017: double (nullable = true)
 |-- 018: double (nullable = true)
 |-- 019: double (nullable = true)
 |-- 020: double (nullable = true)
 |-- 021: double (nullable = true)
 |-- 022: double (nullable = true)
 |-- 023: double (nullable = true)
 |-- 024: double (nullable = true)
 |-- 025: double (nullable = true)
 |-- 026: double (nullable = true)
 |-- 027: double (nullable = true)
 |-- 028: double 

#### Feature columns

In [4]:
featuresCols=covtype_df.columns[:54]
print(featuresCols)

['2596', '51', '3', '258', '04', '510', '221', '232', '148', '6279', '110', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '142', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053']


#### Label columns

In [5]:
covtype_df=covtype_df.withColumn("label", covtype_df["5"]-1).drop("5")
covtype_df.show(1)

+------+----+---+-----+----+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|  2596|  51|  3|  258|  04|  510|  221|  232|  148|  6279|110|011|012|013|014|015|016|017|018|019|020|021|022|023|024|025|026|027|028|029|030|031|032|033|034|035|036|037|038|039|040|041|142|043|044|045|046|047|048|049|050|051|052|053|label|
+------+----+---+-----+----+-----+-----+-----+-----+------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+-----+
|2590.0|56.0|2.0|212.0|-6.0|390.0|220.0|235.0|151.0|6225.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|1.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|0.0|  4.0|
+------+----+---+-----+----+----

#### Train/test splitting

In [6]:
train_df, test_df=covtype_df.randomSplit([0.7, 0.3])
train_df.cache()
test_df.cache()

DataFrame[2596: double, 51: double, 3: double, 258: double, 04: double, 510: double, 221: double, 232: double, 148: double, 6279: double, 110: double, 011: double, 012: double, 013: double, 014: double, 015: double, 016: double, 017: double, 018: double, 019: double, 020: double, 021: double, 022: double, 023: double, 024: double, 025: double, 026: double, 027: double, 028: double, 029: double, 030: double, 031: double, 032: double, 033: double, 034: double, 035: double, 036: double, 037: double, 038: double, 039: double, 040: double, 041: double, 142: double, 043: double, 044: double, 045: double, 046: double, 047: double, 048: double, 049: double, 050: double, 051: double, 052: double, 053: double, label: double]

### Decision tree 

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

vectorAssembler=VectorAssembler(inputCols=featuresCols, outputCol="features")

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features",
                          impurity="gini", maxDepth=5, maxBins=20)

dt_pipeline=Pipeline(stages=[vectorAssembler, dt])
dt_pipeline.getStages()

[VectorAssembler_536f6567f14f, DecisionTreeClassifier_38375228d5ca]

In [8]:
pipelineModel=dt_pipeline.fit(train_df)
pipelineModel.stages[1].toDebugString[:500]

u'DecisionTreeClassificationModel (uid=DecisionTreeClassifier_38375228d5ca) of depth 5 with 39 nodes\n  If (feature 0 <= 3053.5)\n   If (feature 0 <= 2567.5)\n    If (feature 10 <= 0.5)\n     If (feature 0 <= 2398.5)\n      If (feature 3 <= 15.0)\n       Predict: 3.0\n      Else (feature 3 > 15.0)\n       Predict: 2.0\n     Else (feature 0 > 2398.5)\n      Predict: 2.0\n    Else (feature 10 > 0.5)\n     Predict: 1.0\n   Else (feature 0 > 2567.5)\n    If (feature 0 <= 2964.5)\n     If (feature 15 <= 0.5)\n      Pr'

In [9]:
predicted=pipelineModel.transform(test_df)
print(predicted.columns)

['2596', '51', '3', '258', '04', '510', '221', '232', '148', '6279', '110', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039', '040', '041', '142', '043', '044', '045', '046', '047', '048', '049', '050', '051', '052', '053', 'label', 'features', 'rawPrediction', 'probability', 'prediction']


In [10]:
predicted.select("features", "rawPrediction", "probability", "label", "prediction").show(10)
predicted.select("probability", "prediction").take(10)

+--------------------+--------------------+--------------------+-----+----------+
|            features|       rawPrediction|         probability|label|prediction|
+--------------------+--------------------+--------------------+-----+----------+
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  2.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,5,...|[0.0,441.0,11859....|[0.0,0.0241842610...|  5.0|       2.0|
|(54,[0,1,2,3,4,

[Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0),
 Row(probability=DenseVector([0.0, 0.0242, 0.6503, 0.0607, 0.0, 0.2648, 0.0]), prediction=2.0)]

In [11]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", 
                                            metricName="accuracy")

accuracy=evaluator.evaluate(predicted)
accuracy

0.7014970711968546

#### Decision tree with TrainValidation

In [11]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

vectorAssembler=VectorAssembler(inputCols=featuresCols, outputCol="features")

dt=DecisionTreeClassifier(labelCol="label", featuresCol="features",
                          impurity="gini", maxDepth=5, maxBins=20)

paramGrid=ParamGridBuilder().addGrid(dt.impurity, ["gini", "entropy"])\
.addGrid(dt.maxDepth, [10, 15, 20])\
.addGrid(dt.maxBins, [30, 40, 50])\
.build()

evaluator=MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", 
                                            metricName="accuracy")

tvs=TrainValidationSplit(estimator=dt, evaluator=evaluator,\
                         estimatorParamMaps=paramGrid, trainRatio=0.8)

tvs_pipeline=Pipeline(stages=[vectorAssembler, tvs])
tvs_pipelineModel=tvs_pipeline.fit(train_df)

bestModel=tvs_pipelineModel.stages[1].bestModel
bestModel.toDebugString[:500]

u'DecisionTreeClassificationModel (uid=DecisionTreeClassifier_2d0b05f495a2) of depth 20 with 32483 nodes\n  If (feature 0 <= 2697.5)\n   If (feature 0 <= 2471.5)\n    If (feature 23 <= 0.5)\n     If (feature 3 <= 15.0)\n      If (feature 12 <= 0.5)\n       If (feature 5 <= 712.0)\n        If (feature 9 <= 980.0)\n         If (feature 6 <= 211.5)\n          If (feature 14 <= 0.5)\n           If (feature 5 <= 298.5)\n            If (feature 18 <= 0.5)\n             If (feature 8 <= 157.5)\n              If (feat'

In [14]:
predictions=tvs_pipelineModel.transform(test_df)
result=predictions.withColumnRenamed("2596", "海拔")\
.withColumnRenamed("51", "方位")\
.withColumnRenamed("3", "斜率")\
.withColumnRenamed("258", "垂直距離")\
.withColumnRenamed("04", "水平距離")\
.withColumnRenamed("510", "陰影")

result.select("海拔", "方位", "斜率", "垂直距離", "水平距離", "陰影", "label", "prediction").show(10)

+------+----+----+--------+--------+-----+-----+----------+
|  海拔|方位|斜率|垂直距離|水平距離| 陰影|label|prediction|
+------+----+----+--------+--------+-----+-----+----------+
|1863.0|37.0|17.0|   120.0|    18.0| 90.0|  5.0|       5.0|
|1867.0|20.0|15.0|   108.0|    19.0|120.0|  2.0|       2.0|
|1871.0|22.0|22.0|    60.0|    12.0| 85.0|  5.0|       5.0|
|1872.0|27.0|16.0|    95.0|    22.0|124.0|  2.0|       2.0|
|1873.0|30.0|19.0|    67.0|    21.0| 85.0|  2.0|       2.0|
|1879.0|28.0|19.0|    30.0|    12.0| 95.0|  5.0|       5.0|
|1880.0|13.0|23.0|    90.0|    29.0| 67.0|  5.0|       5.0|
|1880.0|35.0|21.0|   150.0|    32.0|150.0|  2.0|       2.0|
|1880.0|38.0|21.0|   150.0|    38.0|120.0|  5.0|       5.0|
|1883.0|27.0|24.0|   120.0|    24.0|108.0|  5.0|       5.0|
+------+----+----+--------+--------+-----+-----+----------+
only showing top 10 rows



In [15]:
accuracy=evaluator.evaluate(predictions)
accuracy

0.9077771275802324